In [1]:
import pandas as pd
import numpy as np
import brightway2 as bw
import wurst
import copy

Set BW2 project

In [2]:
PROJECT_NAME = 'iri'
bw.projects.set_current(PROJECT_NAME)
print(bw.projects.current) # Current project
print(bw.projects.dir) # Current project directory
print(bw.databases) # List of databases in the project

iri
C:\Users\iistrate\AppData\Local\pylca\Brightway3\iri.5753d2a2da40d04ad7f3cc7a024b6e90
Databases dictionary with 16 object(s):
	A_technosphere_flows
	Biogas to chemicals, IMAGE SSP2-Base, 2020
	Biogas-to-chemicals, BAU
	Internet Environmental Footprint
	Internet Environmental Footprint, IMAGE SSP2-RCP19, 2030 (premise v1.2.6)
	Internet Environmental Footprint, IMAGE SSP2-RCP19, 2040 (premise v1.2.6)
	Internet Environmental Footprint, IMAGE SSP2-RCP19, 2050 (premise v1.2.6)
	biosphere3
	ecoinvent 3.1 cutoff
	ecoinvent 3.5 cutoff
	ecoinvent 3.8 cutoff
	ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2030 (premise v1.2.6)
	ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2040 (premise v1.2.6)
	ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2050 (premise v1.2.6)
	ecoinvent 3.8 cutoff, with premise inventories (v1.2.6)
	test


Run basic functions to create new database

In [36]:
%run 00_Functions_to_create_database.ipynb

Set ecoinvent and biosphere databases and import ecoinvent db into wurst format

In [ ]:
biosphere_db = [ef.as_dict() for ef in bw.Database('biosphere3')]

In [57]:
#ei_db = 'ecoinvent 3.8 cutoff'
#ei_db = 'ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2030 (premise v1.2.6)'
#ei_db = 'ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2040 (premise v1.2.6)'
ei_db = 'ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2050 (premise v1.2.6)'

ei_db_wurst = wurst.extract_brightway2_databases([ei_db])

Getting activity data


100%|█████████████████████████████████████████████████████████████████████████| 53223/53223 [00:01<00:00, 48722.02it/s]


Adding exchange data to activities


100%|█████████████████████████████████████████████████████████████████████| 2614582/2614582 [02:46<00:00, 15704.47it/s]


Filling out exchange data


100%|██████████████████████████████████████████████████████████████████████████| 53223/53223 [00:10<00:00, 4956.67it/s]


# Import inventories for ICT sub-systems

In [58]:
LCI_file = "LCI_DigitalContentConsumption.xlsx"

DS_ICT_raw = bw.ExcelImporter(LCI_file)
DS_ICT_raw.apply_strategies()
DS_ICT_raw.match_database(ei_db, fields=('name', 'reference product', 'unit', 'location'))
DS_ICT_raw.match_database("biosphere3", fields=('name', 'unit', 'categories'))
DS_ICT_raw.match_database(fields=('name', 'reference product', 'unit', 'location'))
DS_ICT_raw.statistics()

Extracted 1 worksheets in 0.15 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.22 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
5 datasets
29 exchanges
0 unlinked exchanges

(5, 29, 0)

In [59]:
# Change reference product to product
DS_ICT = copy.deepcopy(DS_ICT_raw.data)
DS_ICT = correct_reference_product(DS_ICT)

# Import inventories for digital content

In [60]:
COL_START = 7

LCI_DigitalContent = pd.read_excel(LCI_file, sheet_name='LCI_DigitalContent', skiprows=1).fillna(0)
NO_COLS = 7
Data_Traffic = pd.read_excel(LCI_file, sheet_name='Data_Traffic', skiprows=1, index_col=0)

DS_DigitalContent = create_datasets_from_df(LCI_DigitalContent, Data_Traffic, NO_COLS, COL_START, DS_ICT + ei_db_wurst, biosphere_db)

# Import inventories for users

In [61]:
LCI_Users = pd.read_excel(LCI_file, sheet_name='LCI_Users', skiprows=1).fillna(0)
NO_COLS = 3

DS_Users = create_datasets_from_df(LCI_Users, Data_Traffic, NO_COLS, COL_START, DS_ICT + DS_DigitalContent + ei_db_wurst, biosphere_db)

In [62]:
DigitalContentConsumption_db = []
DigitalContentConsumption_db = DS_ICT + DS_DigitalContent + DS_Users

# Create regional datasets

In [41]:
GLO_ds = ['market for data centers, internet access',
          'core network, internet access, data centers node']
ds_to_regionalize = [ds for ds in DigitalContentConsumption_db if ds['name'] not in GLO_ds]

# Creating datasets for all the locations with an electricity market in ecoinvent (187) 
# would require substantial computational efforts. Consequently, we selected a number of 
# locations based on electricity carbon footprint
new_locations = {# Regions
                 'BR': 'Brazil', 
                 'CA': 'Canada',
                 'CN': 'China',
                 'IN': 'India',
                 'RER': 'Europe',
                 'US': 'United States',
                 # Europe
                 'NO': 'Norway',
                 'CH': 'Switzerland',
                 'FR': 'France',
                 'DK': 'Denmark',
                 'ES': 'Spain',
                 'IT': 'Italy',
                 'DE': 'Germany',
                 'PL': 'Poland',
                 'GB': 'United Kingdom',
                 # Other countries
                 'ZA': 'South Africa',
                 'AU': 'Australia',
                 'RU': 'Russia'
                 }

# Create datasets to the new locations
DS_new_locs = create_datasets_for_locations(ds_to_regionalize, new_locations, DigitalContentConsumption_db + ei_db_wurst, biosphere_db)

# Link database and write

In [42]:
new_db_name = 'Digital Content Consumption'
if new_db_name in bw.databases:
    del bw.databases[new_db_name]
wurst.write_brightway2_database(DigitalContentConsumption_db + DS_new_locs, new_db_name)

914 datasets
7945 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/28/2022 12:06:43
  Finished: 09/28/2022 12:06:43
  Total time elapsed: 00:00:00
  CPU %: 82.50
  Memory %: 34.41
Created database: Digital Content Consumption


In [63]:
# Prospective database only for GLO location
#new_db_name = 'Digital Content Consumption, IMAGE SSP2-RCP19, 2030 (premise v1.2.6)'
#new_db_name = 'Digital Content Consumption, IMAGE SSP2-RCP19, 2040 (premise v1.2.6)'
new_db_name = 'Digital Content Consumption, IMAGE SSP2-RCP19, 2050 (premise v1.2.6)'
if new_db_name in bw.databases:
    del bw.databases[new_db_name]
wurst.write_brightway2_database(DigitalContentConsumption_db, new_db_name)

50 datasets
439 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/28/2022 14:25:21
  Finished: 09/28/2022 14:25:21
  Total time elapsed: 00:00:00
  CPU %: 71.70
  Memory %: 59.59
Created database: Digital Content Consumption, IMAGE SSP2-RCP19, 2050 (premise v1.2.6)


In [14]:
for a in bw.Database("DEF_db_name"):
    assert len(a.production()) == 1

In [22]:
for ds in DEF_db_linked:
    for exc in ds['exchanges']:
        if exc['type'] == 'production':
            if ds['reference product'] != exc['product']:
                print('yes')